******DON'T FORGET TO UPVOTE******

In [ ]:

import matplotlib.pyplot as plt
import imageio
import matplotlib.image as img
from tensorflow.keras.preprocessing import image
import numpy as np

import pathlib
from pathlib import Path



from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.models import load_model
from PIL import Image

import tensorflow as tf
from tensorflow.keras import Sequential
from keras.layers import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_path=pathlib.Path('/kaggle/input/yoga-16/yoga16-dataset/train')
test_path=pathlib.Path('/kaggle/input/yoga-16/yoga16-dataset/test')
val_path=pathlib.Path('/kaggle/input/yoga-16/yoga16-dataset/val')

In [ ]:
image_classes=os.listdir(train_path)
image_classes

In [ ]:
def plot_images (image_classes):
    
    plt.figure(figsize=(12,12))
    
    for i, category in enumerate (image_classes):
        
        image_path= train_path / category
        image_in_folder=os.listdir(image_path)
        
        first_image=image_in_folder[0]
        first_image_path=image_path / first_image
        
        img=image.load_img(first_image_path)
        img_array=image.img_to_array(img) / 255
        
        plt.subplot(4,4, i+1)
        plt.imshow(img_array)
        plt.title(category)
        plt.axis('off')
        
plt.show()    

In [ ]:
plot_images(image_classes)

In [ ]:
image_generator=ImageDataGenerator(rescale=1/255,validation_split=0.2)

In [ ]:
train_data=image_generator.flow_from_directory(train_path,
                                               target_size=(224,224),
                                              batch_size=32,
                                              shuffle=True,
                                              class_mode='categorical',
                                              subset='training')


In [ ]:
test_data=image_generator.flow_from_directory(train_path,
                                               target_size=(224,224),
                                              batch_size=1,
                                              shuffle=False,
                                              class_mode='categorical',
                                              subset='validation')

In [ ]:
print(test_data.image_shape)
print(train_data.image_shape)

In [ ]:
model = Sequential()

model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', input_shape=[224, 224, 3]))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))  
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())

model.add(Dense(128, activation='relu'))  
model.add(Dropout(0.5))  

model.add(Dense(16, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
model_history=model.fit(train_data,epochs=10,validation_data=test_data)

In [ ]:
plt.plot(model_history.history['loss'],label='train_loss')
plt.plot(model_history.history['val_loss'],label='validation_loss')

In [ ]:
plt.plot(model_history.history['accuracy'],label='train_accuracy')
plt.plot(model_history.history['val_accuracy'],label='validation_accuracy')

In [ ]:
model.evaluate(train_data)

In [ ]:
model.evaluate(test_data)

In [ ]:
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)  
actual_classes = test_data.classes             


class_labels = list(test_data.class_indices.keys())  # Class names

for i in range(2):
    img, label = test_data[i] 
    actual_label = class_labels[actual_classes[i]]
    predicted_label = class_labels[predicted_classes[i]]

    plt.imshow(img[0])
    plt.title(f"Actual: {actual_label}, Predicted: {predicted_label}")
    plt.axis('off')
    plt.show()